 # [이산형확률변수의 가설함수 검증 코드짜기]
21102370 유다나

빅데이터(31001)/2023년 1학기

Week 4 assignment

# 데이터 준비


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/3-1/BigData

/content/drive/MyDrive/3-1/BigData


In [3]:
import numpy as np
import pandas as pd

In [4]:
df = pd.read_csv('ncsoft_stock.csv')
df.head()

,Date,Open,High,Low,Close,Volume
0,2020-01-02,542000,545000,539000,541000,40246
1,2020-01-03,547000,568000,542000,565000,112404
2,2020-01-06,562000,587000,562000,579000,107006
3,2020-01-07,583000,596000,574000,594000,84378
4,2020-01-08,587000,604000,584000,604000,109267


In [5]:
data = df.copy()
data.head()

,Date,Open,High,Low,Close,Volume
0,2020-01-02,542000,545000,539000,541000,40246
1,2020-01-03,547000,568000,542000,565000,112404
2,2020-01-06,562000,587000,562000,579000,107006
3,2020-01-07,583000,596000,574000,594000,84378
4,2020-01-08,587000,604000,584000,604000,109267


# 데이터 전처리


In [6]:
# 날짜를 요일로 만들기
from datetime import datetime, date

In [7]:
# # 두번 다시 함수 만들지 않겠습니다...........
# def change_day(date):
#   day = list(map(int, date.split('-')))
#   result = date(day[0], day[1], day[2]).weekday()
#   return result

In [8]:
# 날짜를 요일로 만들기
data['Day_week'] = pd.to_datetime(data['Date'])
data['Day_week'] = data['Day_week'].dt.weekday
data.head()

,Date,Open,High,Low,Close,Volume,Day_week
0,2020-01-02,542000,545000,539000,541000,40246,3
1,2020-01-03,547000,568000,542000,565000,112404,4
2,2020-01-06,562000,587000,562000,579000,107006,0
3,2020-01-07,583000,596000,574000,594000,84378,1
4,2020-01-08,587000,604000,584000,604000,109267,2


In [9]:
# 전날꺼랑 비교해서 상승 하락 추가하기
data['Stock_last'] = data['Close'].shift(1)
data.head()

,Date,Open,High,Low,Close,Volume,Day_week,Stock_last
0,2020-01-02,542000,545000,539000,541000,40246,3,NaN
1,2020-01-03,547000,568000,542000,565000,112404,4,541000.0
2,2020-01-06,562000,587000,562000,579000,107006,0,565000.0
3,2020-01-07,583000,596000,574000,594000,84378,1,579000.0
4,2020-01-08,587000,604000,584000,604000,109267,2,594000.0


In [10]:
data['Stock_Up'] = data['Close'] > data['Stock_last']
data

,Date,Open,High,Low,Close,Volume,Day_week,Stock_last,Stock_Up
0,2020-01-02,542000,545000,539000,541000,40246,3,NaN,False
1,2020-01-03,547000,568000,542000,565000,112404,4,541000.0,True
2,2020-01-06,562000,587000,562000,579000,107006,0,565000.0,True
3,2020-01-07,583000,596000,574000,594000,84378,1,579000.0,True
4,2020-01-08,587000,604000,584000,604000,109267,2,594000.0,True
...,...,...,...,...,...,...,...,...,...
558,2022-04-06,475000,483500,469000,474000,96750,2,480000.0,False
559,2022-04-07,469000,471500,462000,462000,103079,3,474000.0,False
560,2022-04-08,462500,466500,453500,458000,87055,4,462000.0,False
561,2022-04-11,450500,459000,449000,453500,59322,0,458000.0,False


In [11]:
# 주가가 오른 날만 확인
data_up = data[data['Stock_Up']==True]
data_up.head()

,Date,Open,High,Low,Close,Volume,Day_week,Stock_last,Stock_Up
1,2020-01-03,547000,568000,542000,565000,112404,4,541000.0,True
2,2020-01-06,562000,587000,562000,579000,107006,0,565000.0,True
3,2020-01-07,583000,596000,574000,594000,84378,1,579000.0,True
4,2020-01-08,587000,604000,584000,604000,109267,2,594000.0,True
5,2020-01-09,597000,608000,596000,608000,71171,3,604000.0,True


# 관측값을 토대로 확률분포 그리기



In [12]:
# 확률분포 그리기
week_day = [i for i in range(5)]

def count_day(x):
  return len(data_up[data_up['Day_week']==x])
Up_day = np.array([count_day(x) for x in week_day])

print(Up_day)

[46 51 64 52 39]


In [13]:
Up_total = len(data_up)
prob = np.array([day/Up_total for day in Up_day])
dict(zip(week_day, prob))

{0: 0.18253968253968253,
 1: 0.20238095238095238,
 2: 0.25396825396825395,
 3: 0.20634920634920634,
 4: 0.15476190476190477}

In [14]:
np.sum(prob)

0.9999999999999999

# 관측값을 토대로 확률 함수 구현


In [15]:
def f(x):
  if x in week_day:
    return prob[x]
  else:
    return 0

In [16]:
f(1)

0.20238095238095238

#reference
-[shift fun](https://bskyvision.com/entry/pandas-%EC%9D%B4%EC%A0%84-%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%99%80-%EB%B9%84%EA%B5%90%ED%95%A0-%EB%95%8C-%EC%9C%A0%EC%9A%A9%ED%95%9C-%EB%8D%B0%EC%9D%B4%ED%84%B0%ED%94%84%EB%A0%88%EC%9E%84-shift-%EB%A9%94%EC%86%8C%EB%93%9C)

-[비교 fun](https://pinggoopark.tistory.com/507)

-[pandas1](https://computer-science-student.tistory.com/375)

